In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<div style="position: relative; text-align: center; color: orange;">
  <img src="https://www.thehousedesigners.com/images/plans/01/AEA/uploads/main-image_white-lily_m.webp" alt="Notebook Intro" style="width:100%;">
  
  <!-- Welcome to My Notebook Text -->
  <div style="position: absolute; top: 10%; left: 26%; transform: translate(-50%, -50%); font-weight: bold; font-size: 50px; text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.7);">
    Welcome to My Notebook
  </div>
  
  <!-- Sahil Khan Text -->
  <div style="position: absolute; top: 82%; left: 40%; transform: translate(-50%, -50%); font-weight: bold; font-size: 50px; text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.7);">
    Sahil Khan
  </div>
</div>


<h1 style="
    display: inline-block;
    color: black; 
    background-color: orange; 
    padding: 10px 20px; 
    border-radius: 8px; 
    box-shadow: 3px 3px 8px rgba(0, 0, 0, 0.3);
    text-align: center;
    font-size: 20px;
    font-weight: bold;
    margin: 0;
">
    Introduction
</h1>

#### The dataset consists of 79 explanatory variables (features) that describe various aspects of residential homes in Ames, Iowa. The target variable is the SalePrice, which represents the sale price of each house.

In [ ]:
# you can review the columns of the dataset by running the following this cell

# with open("/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt") as f:
#     data_desc = f.read()

# print(data_desc)

---

<h1 style="
    display: inline-block;
    color: black; 
    background-color: orange; 
    padding: 10px 20px; 
    border-radius: 8px; 
    box-shadow: 3px 3px 8px rgba(0, 0, 0, 0.3);
    text-align: center;
    font-size: 20px;
    font-weight: bold;
    margin: 0;
">
    Importing the Required libraries
</h1>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer

from sklearn.ensemble import RandomForestRegressor, StackingRegressor, GradientBoostingRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline, Pipeline

import warnings
# Suppress the specific FutureWarning
warnings.filterwarnings("ignore", category=FutureWarning, module="seaborn")

<h1 style="
    display: inline-block;
    color: black; 
    background-color: orange; 
    padding: 10px 20px; 
    border-radius: 8px; 
    box-shadow: 3px 3px 8px rgba(0, 0, 0, 0.3);
    text-align: center;
    font-size: 20px;
    font-weight: bold;
    margin: 0;
">
    Loading the Dataset
</h1>


In [ ]:
sample_submission = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv")
train_df = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
test_df = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")

In [ ]:
# Setting the max column to 100 
pd.set_option('display.max_columns', 100)

In [ ]:
train_df.shape, test_df.shape

In [ ]:
train_df

In [ ]:
train_df.info()

In [ ]:
train_df.describe()

<h1 style="
    display: inline-block;
    color: black; 
    background-color: orange; 
    padding: 10px 20px; 
    border-radius: 8px; 
    box-shadow: 3px 3px 8px rgba(0, 0, 0, 0.3);
    text-align: center;
    font-size: 20px;
    font-weight: bold;
    margin: 0;
">
    Data Preprocessing
</h1>

In [ ]:
train_df.isnull().sum()

In [ ]:
# Percentage of missing values of each column
len_of_df = len(train_df)
miss_value_df = pd.DataFrame({
    'Miss Count':train_df.isnull().sum().values, 
    'Miss %': 100 * (train_df.isnull().sum().values / len_of_df)}, 
    index = train_df.isnull().sum().index
)

missing_value_df = miss_value_df.sort_values(by = ['Miss %'], ascending = False)
missing_value_df.head(15)

In [ ]:
# names of columns where missing value is greater than 20%

cols_to_drop = missing_value_df[missing_value_df['Miss %'] > 20].index
cols_to_drop

In [ ]:
# Dropping columns 
for col in cols_to_drop:
    train_df.drop(col, axis = 1, inplace = True)
    test_df.drop(col, axis = 1, inplace = True)    

In [ ]:
train_df.shape, test_df.shape

<h1 style="
    display: inline-block;
    color: black; 
    background-color: orange; 
    padding: 10px 20px; 
    border-radius: 8px; 
    box-shadow: 3px 3px 8px rgba(0, 0, 0, 0.3);
    text-align: center;
    font-size: 20px;
    font-weight: bold;
    margin: 0;
">
    Visualization :
</h1>

In [ ]:
# Replacing the inf values with NaN
train_df.replace([float('inf'), -float('inf')], pd.NA, inplace=True)
test_df.replace([float('inf'), -float('inf')], pd.NA, inplace=True)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot histogram of LotFrontage
sns.histplot(train_df['LotFrontage'], binwidth=2, ax=axes[0])
axes[0].set_title('Histogram of LotFrontage')

# Plot scatterplot of LotFrontage vs SalePrice, highlighting missing values
sns.scatterplot(x=train_df['LotFrontage'], y=train_df['SalePrice'], label='Complete values', ax=axes[1])
missing_LotFrontage = train_df[train_df['LotFrontage'].isna()]
sns.scatterplot(x=[0]*len(missing_LotFrontage), y=missing_LotFrontage['SalePrice'], color='red', label='Missing values', ax=axes[1])
axes[1].legend()
axes[1].set_title('Scatterplot of LotFrontage vs SalePrice')

# Adjust layout
plt.tight_layout()

<h2 style="
    display: inline-block;
    color: black; 
    background-color: orange; 
    padding: 10px 20px; 
    border-radius: 8px; 
    box-shadow: 3px 3px 8px rgba(0, 0, 0, 0.3);
    text-align: center;
    font-size: 20px;
    font-weight: bold;
    margin: 0;
">
    Visualizing Target Column
</h2>

In [ ]:
# Checking the skewness of the dataset
print(f"Skewness: { train_df['SalePrice'].skew() }")
sns.histplot(x = train_df['SalePrice'],kde=True)

In [ ]:
# Log transformation
sns.histplot(x = np.log(train_df['SalePrice']), kde = True)

<h2 style="
    display: inline-block;
    color: black; 
    background-color: lightblue; 
    padding: 10px 20px; 
    border-radius: 8px; 
    box-shadow: 3px 3px 8px rgba(0, 0, 0, 0.3);
    text-align: center;
    font-size: 20px;
    font-weight: bold;
    margin: 0;
">
    Visualizing Numeric Columns
</h2>

In [ ]:
# Selecting Numerical Columns

num_features_cols = train_df.drop('Id', axis = 1).select_dtypes(include=['number']).columns

# Creating Distribution of Numerical Features 

for col in num_features_cols:
    
    plt.figure(figsize=(15, 4))
    
    plt.subplot(1, 2, 1)
    sns.histplot(train_df[col], kde = True)
    plt.subplot(1, 2, 2)
    sns.scatterplot(data = train_df, x = col, y='SalePrice')

<h2 style="
    display: inline-block;
    color: black; 
    background-color: lightblue; 
    padding: 10px 20px; 
    border-radius: 8px; 
    box-shadow: 3px 3px 8px rgba(0, 0, 0, 0.3);
    text-align: center;
    font-size: 20px;
    font-weight: bold;
    margin: 0;
">
    Visualizing Categorical Columns
</h2>

In [ ]:
# Selecting Categorical features
cat_features_cols = train_df.select_dtypes(exclude=['number']).columns

# Creating Distribution of Categorical Features 
for col in cat_features_cols:
    
    plt.figure(figsize=(15, 4))
    
    plt.subplot(1, 2, 1)
    sns.histplot(train_df[col], kde = True)
    plt.subplot(1, 2, 2)
    sns.scatterplot(data = train_df, x = col, y='SalePrice')

In [ ]:
numeric_columns = train_df.select_dtypes(include=['number'])
corr_df = numeric_columns.corr()
corr_df

In [ ]:
correlations = corr_df.corr()[['SalePrice']].sort_values(by='SalePrice', ascending=False)

# Plot heatmap of correlations
plt.figure(figsize=(10, 10))
sns.heatmap(
    correlations,
    annot=True,
    cmap='coolwarm',
    fmt=".2f",
    vmin=-1,
    vmax=1
)
plt.title('Correlation of Numeric Features with SalePrice')

<h1 style="
    display: inline-block;
    color: black; 
    background-color: orange; 
    padding: 10px 20px; 
    border-radius: 8px; 
    box-shadow: 3px 3px 8px rgba(0, 0, 0, 0.3);
    text-align: center;
    font-size: 20px;
    font-weight: bold;
    margin: 0;
">
    Feature Engineering :
</h1>

In [ ]:
# Creating New Features
train_df['HouseAge'] = train_df['YrSold'] - train_df['YearBuilt']
train_df['RemodAge'] = train_df['YrSold'] - train_df['YearRemodAdd']
test_df['HouseAge'] = test_df['YrSold'] - test_df['YearBuilt']
test_df['RemodAge'] = test_df['YrSold'] - test_df['YearRemodAdd']

train_df['TotalBathrooms'] = (train_df['FullBath'] + (0.5 * train_df['HalfBath']) + train_df['BsmtFullBath'] + (0.5 * train_df['BsmtHalfBath']))
test_df['TotalBathrooms'] = (test_df['FullBath'] + (0.5 * test_df['HalfBath']) + test_df['BsmtFullBath'] + (0.5 * test_df['BsmtHalfBath']))

train_df['TotalPorchSF'] = (train_df['OpenPorchSF'] + train_df['3SsnPorch'] + train_df['EnclosedPorch'] + train_df['ScreenPorch'] + train_df['WoodDeckSF'])
test_df['TotalPorchSF'] = (test_df['OpenPorchSF'] + test_df['3SsnPorch'] + test_df['EnclosedPorch'] + test_df['ScreenPorch'] + test_df['WoodDeckSF'])

train_df['TotalRooms'] = train_df['TotRmsAbvGrd'] + train_df['TotalBsmtSF']
test_df['TotalRooms'] = test_df['TotRmsAbvGrd'] + test_df['TotalBsmtSF']

<h2 style="
    display: inline-block;
    color: black; 
    background-color: red; 
    padding: 10px 20px; 
    border-radius: 8px; 
    box-shadow: 3px 3px 8px rgba(0, 0, 0, 0.3);
    text-align: center;
    font-size: 20px;
    font-weight: bold;
    margin: 0;
">
    Dropping Columns
</h2>
<br></br>

After visualizting and reading the data description the whole, i have selected some columns based on correlation and multi-colinearity, which will not contribute that much in prediction, but together will reduce the performance of our model.

So i am simply dropping them. 

In [ ]:
cols_to_drop = ['Id', 'OverallCond', 'YrSold', 'YearBuilt', 'YearRemodAdd', 'GarageCars', 'GarageCond', 'Condition2', 'BsmtHalfBath', 'BsmtFullBath', 'HalfBath', 'FullBath', 'WoodDeckSF', 'ScreenPorch', 'EnclosedPorch', '3SsnPorch', 'OpenPorchSF', 'TotalBsmtSF', 'TotRmsAbvGrd', 'MoSold', 'MiscVal', 'GarageYrBlt', 'LowQualFinSF', 'MasVnrArea', '2ndFlrSF', '1stFlrSF', 'BsmtUnfSF', 'BsmtFinSF2', 'BsmtFinSF1', 'GarageFinish', 'BsmtFinType1', 'BsmtFinType2', 'BsmtCond', 'ExterCond', 'RoofMatl', 'RoofStyle', 'Exterior2nd', 'PoolArea']

train_df.drop(columns = cols_to_drop, axis = 1, inplace=True)
test_df.drop(columns = cols_to_drop, axis = 1, inplace=True)

In [ ]:
train_df.shape, test_df.shape

In [ ]:
# Again checking the correlation
a_numeric_columns = train_df.select_dtypes(include=['number'])
a_corr_df = a_numeric_columns.corr()
a_corr_df

In [ ]:
correlations = a_corr_df.corr()[['SalePrice']].sort_values(by='SalePrice', ascending=False)

# Plot heatmap of correlations
plt.figure(figsize=(10, 10))
sns.heatmap(
    correlations,
    annot=True,
    cmap='coolwarm',
    fmt=".2f",
    vmin=-1,
    vmax=1
)
plt.title('Correlation of Numeric Features with SalePrice')

Still `BedroomAbvGr` and `MSSubClass` are not that much affecting the SalePrice, so i will remove two more columns.

In [ ]:
train_df.drop(['BedroomAbvGr', 'MSSubClass'], axis = 1, inplace = True)
test_df.drop(['BedroomAbvGr', 'MSSubClass'], axis = 1, inplace = True)

<h2 style="
    display: inline-block;
    color: black; 
    background-color: orange; 
    padding: 10px 20px; 
    border-radius: 8px; 
    box-shadow: 3px 3px 8px rgba(0, 0, 0, 0.3);
    text-align: center;
    font-size: 20px;
    font-weight: bold;
    margin: 0;
">
    Handling Missing values of Seleted Features
</h2>

In [ ]:
# train_df.isnull().sum().sort_values(ascending=False)

In [ ]:
# test_df.isnull().sum().sort_values(ascending=False)

Filling the missing values of categorical columns with most frequent values, and numeric columns with median

In [ ]:
numeric_cols = train_df.drop('SalePrice', axis = 1).select_dtypes(include=['number']).columns
categorical_cols = train_df.select_dtypes(exclude=['number']).columns

# Filling missing values of numeric cols
for index, col in enumerate(numeric_cols):
    median = train_df[col].median()
    train_df[col] = train_df[col].fillna(median)
    test_df[col] = test_df[col].fillna(median)


# Filling missing values of categorical cols
for index, col in enumerate(categorical_cols):
    most_frequent = train_df[col].value_counts().index[0]
    train_df[col] = train_df[col].fillna(most_frequent)
    test_df[col] = test_df[col].fillna(most_frequent)

In [ ]:
train_df.isnull().sum().sort_values(ascending=False)

In [ ]:
test_df.isnull().sum().sort_values(ascending=False)

<h1 style="
    display: inline-block;
    color: black; 
    background-color: orange; 
    padding: 10px 20px; 
    border-radius: 8px; 
    box-shadow: 3px 3px 8px rgba(0, 0, 0, 0.3);
    text-align: center;
    font-size: 20px;
    font-weight: bold;
    margin: 0;
">
    Encoding the features
</h1>

In [ ]:
# First removing the less important features from the remaining dataset

less_imp_cols = ['LandContour', 'Street', 'Electrical', 'PavedDrive', 'BsmtExposure', 'Functional', 'GarageQual']

train_df.drop(columns = less_imp_cols, axis = 1, inplace = True)
test_df.drop(columns = less_imp_cols, axis = 1, inplace = True)

In [ ]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder

# Create mappings for ordinal categories
ordinal_cols = ['ExterQual', 'BsmtQual', 'HeatingQC', 'KitchenQual', 'LotShape', 'LandSlope']
ordinal_mapping = {
    'ExterQual': {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
    'BsmtQual': {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
    'HeatingQC': {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
    'KitchenQual': {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
    'LotShape': {'IR3': 1, 'IR2': 2, 'IR1': 3, 'Reg': 4},
    'LandSlope': {'Sev': 1, 'Mod': 2, 'Gtl': 3}
}

# Apply the ordinal encoding
for col, mapping in ordinal_mapping.items():
    train_df[col] = train_df[col].map(mapping)
    test_df[col] = test_df[col].map(mapping)

# Check the transformed columns
train_df[ordinal_cols].head()

In [ ]:
# List of categorical columns for One-Hot Encoding
ohe_cols = ['MSZoning', 'Utilities', 'LotConfig', 'Neighborhood',
            'Condition1', 'BldgType', 'HouseStyle', 'Exterior1st', 'Foundation', 'Heating',
            'CentralAir', 'GarageType', 'SaleType', 'SaleCondition']

# One-hot encoding on training data
train_encoded = pd.get_dummies(train_df, dtype='int', columns=ohe_cols, drop_first=True)

# Get the columns that appear after one-hot encoding in training data
ohe_cols_final = train_encoded.columns

# Apply one-hot encoding to testing data
test_encoded = pd.get_dummies(test_df, dtype = 'int', columns=ohe_cols, drop_first=True)

# Align test data with training data columns to avoid issues with missing columns
test_encoded = test_encoded.reindex(columns=ohe_cols_final, fill_value=0)

In [ ]:
train_encoded

<h1 style="
    display: inline-block;
    color: black; 
    background-color: orange; 
    padding: 10px 20px; 
    border-radius: 8px; 
    box-shadow: 3px 3px 8px rgba(0, 0, 0, 0.3);
    text-align: center;
    font-size: 20px;
    font-weight: bold;
    margin: 0;
">
    Model Building
</h1>


***The thing you are waiting for***

<h2 style="
    display: inline-block;
    color: black; 
    background-color: orange; 
    padding: 10px 20px; 
    border-radius: 8px; 
    box-shadow: 3px 3px 8px rgba(0, 0, 0, 0.3);
    text-align: center;
    font-size: 20px;
    font-weight: bold;
    margin: 0;
">
    Splitting the data
</h2>

In [ ]:
X = train_encoded.drop('SalePrice', axis = 1)
y = train_encoded['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

In [ ]:
# Building the model

import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error

# Initialize models
models = {
    'DecisionTree': DecisionTreeRegressor(random_state=42),
    'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42),
    'XGBoost': xgb.XGBRegressor(objective='reg:squarederror', random_state=42),
    'LightGBM': lgb.LGBMRegressor(random_state=42),
    'CatBoost': CatBoostRegressor(verbose=0, random_state=42)
}

# Initialize an empty list to store results
results = []

# Train each model, predict, and calculate RMSE and accuracy
for name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Calculate RMSE
    mse = mean_squared_error(y_test, y_pred)
    rmse = mse ** 0.5
    
    # Calculate Accuracy (R^2 score)
    accuracy = model.score(X_test, y_test)
    
    # Store the result in a dictionary and append to the list
    results.append({'Model': name, 'RMSE': rmse, 'Accuracy (R2 Score)': accuracy})

# Convert the results list to a DataFrame
results_df = pd.DataFrame(results)

# Display the results DataFrame
print("Model Performance:")
results_df

In [ ]:
# Identify the best model based on highest accuracy (R2 Score)
best_model_name = results_df.loc[results_df['Accuracy (R2 Score)'].idxmax(), 'Model']
best_model = models[best_model_name]

print(f"\nBest Model: {best_model_name}")

# Retrain the best model on the full training dataset (if needed)
best_model.fit(X_train, y_train)

In [ ]:
y_pred = best_model.predict(test_encoded)

In [ ]:
sample_submission

In [ ]:
y_pred

<h2 style="
    display: inline-block;
    color: black; 
    background-color: orange; 
    padding: 10px 20px; 
    border-radius: 8px; 
    box-shadow: 3px 3px 8px rgba(0, 0, 0, 0.3);
    text-align: center;
    font-size: 20px;
    font-weight: bold;
    margin: 0;
">
    Hyper-Parameter Tuning
</h2>


In [ ]:
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

# Prepare the data
X = train_encoded.drop('SalePrice', axis=1)
y = train_encoded['SalePrice']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

# Initialize the CatBoostRegressor
catboost_model = CatBoostRegressor(verbose=0, random_state=42)

# Define the parameter grid
param_grid = {
    'iterations': [100, 200, 300],
    'depth': [6, 8, 10],
    'learning_rate': [0.01, 0.05, 0.1],
    'l2_leaf_reg': [1, 3, 5],
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=catboost_model, param_grid=param_grid, 
                           cv=3, scoring='neg_mean_squared_error', 
                           n_jobs=-1, verbose=1)

# Perform Grid Search
grid_search.fit(X_train, y_train)

# Get the best model from grid search
best_catboost_model = grid_search.best_estimator_

# Make predictions with the best model
y_pred = best_catboost_model.predict(X_test)

# Calculate RMSE
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5

# Get the best parameters and print results
best_params = grid_search.best_params_

print(f"Best Parameters: {best_params}")
print(f"Best RMSE: {rmse}")

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_pred, y_test)

<h1 style="
    display: inline-block;
    color: black; 
    background-color: orange; 
    padding: 10px 20px; 
    border-radius: 8px; 
    box-shadow: 3px 3px 8px rgba(0, 0, 0, 0.3);
    text-align: center;
    font-size: 20px;
    font-weight: bold;
    margin: 0;
">
    Submission
</h1>

In [ ]:
predictions = best_catboost_model.predict(test_encoded)

submission = pd.DataFrame({'Id':sample_submission['Id'], 'SalePrice':np.round(predictions, 4)})
submission.to_csv('submission.csv', index = False)

In [ ]:
submission

<h2>😎😎</h2>
<div style="display: flex; justify-content: space-evenly; margin-top: 20px;">
    <a href="https://www.linkedin.com/in/programmer70" style="background-color:#0077b5; color:white; padding:10px 20px; text-decoration:none; border-radius:5px;">Connect on LinkedIn</a>
    <a href="https://www.kaggle.com/sahilkhan70" style="background-color:#20beff; color:white; padding:10px 20px; text-decoration:none; border-radius:5px;">Kaggle Profile</a>
    <a href="mailto:sahilkhan782466@gmail.com" style="background-color:#c71610; color:white; padding:10px 20px; text-decoration:none; border-radius:5px;">Email Me</a>
    <a href="https://github.com/sahilkhan-7" style="background-color:#333; color:white; padding:10px 20px; text-decoration:none; border-radius:5px;">GitHub Profile</a>
    <a href="https://www.instagram.com/programmer.70" style="background-color:#e4405f; color:white; padding:10px 20px; text-decoration:none; border-radius:5px;">Instagram</a>
</div>

In [ ]:
sample_submission

---